In [23]:
import json
from IPython.core.display import display, HTML
import requests
from bs4 import BeautifulSoup
import io


In [63]:
journal='jmis'
alt_detail='https://www.altmetric.com/details/'

In [64]:
with open('altmetrics/'+journal+'_alt.json') as json_data:
    d = json.loads(json_data.read())
    json_data.close()

In [65]:
alt=d

In [66]:
with open('policy/'+journal+'.txt', 'a') as out:
    out.write('doi \t article_title \t policy_source \t policy_date \t policy_doc_title \t policy_doc_link \n')

for doi in alt:
    keys=alt[doi].keys()
    if 'cited_by_policies_count' in keys:
        alt_policy=alt_detail+str(alt[doi]['altmetric_id'])+'/policy-documents'
        HTML("<a href="+alt_policy+" target='_blank'>link</a>")
        r = requests.get(alt_policy)
        soup = BeautifulSoup(r.text,"html5lib")
        for item in soup.find_all('div',{'class':"content with_image"}):
            with io.open('policy/'+journal+'.txt', 'a',encoding="utf-8") as out:
                article_title=alt[doi]['title']
                policy_source=item.find('h4').text.split(' on ')[0].split('Cited by ')[1]
                policy_date=item.find('h4').text.split(' on ')[1][-4:]
                policy_doc_link=item.find().attrs['href']
                policy_doc_title=item.find('h3').text
                line=doi+'\t'+article_title+'\t'+policy_source+'\t'+policy_date+'\t'+policy_doc_title+'\t'+policy_doc_link
                line=line.replace('\n','')
                out.write(line+'\n')

In [67]:
with open('articles/'+journal+'.txt', 'a') as out:
    out.write('doi\t Authors\t Year\t Title\t Journal\n')

for doi in alt:
    keys=alt[doi].keys()
    if 'cited_by_policies_count' in keys:
        crossref_api='http://api.crossref.org/works/'+doi
        r = requests.get(crossref_api)
        soup = BeautifulSoup(r.text,"html5lib")
        message=json.loads(soup.text)['message']
        title=message['title'][0]
        year_online=year_print=9999
        if 'published-online' in message:
            year_online=message['published-online']['date-parts'][0][0]
        if 'published-print' in message:
            year_print=message['published-print']['date-parts'][0][0]
        year=min(year_online,year_print)
        if 'author' in message:
            authors=message['author']
            author_string=authors[0]['family']
            if len(authors)==2:
                author_string=author_string+' & ' + authors[1]['family']
            if len(authors)>3:
                author_string=author_string+' et al.'
        else: 
            author_string='MISSING'
        journal_title=message['container-title'][0]
        with io.open('articles/'+journal+'.txt', 'a',encoding="utf-8") as out:
            line=doi+'\t'+author_string+'\t'+str(year)+'\t'+title+'\t'+journal_title
            out.write(line+'\n')

In [68]:
import fileinput
import glob

file_list = glob.glob("policy/*.txt")

with open('policy_all.txt', 'w') as file:
    input_lines = fileinput.input(file_list)
    file.writelines(input_lines)

In [69]:
import fileinput
import glob

file_list = glob.glob("articles/*.txt")

with open('articles_all.txt', 'w') as file:
    input_lines = fileinput.input(file_list)
    file.writelines(input_lines)